In [1]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 89.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.5
    Not uninstalling pandas at /shared-libs/python3.7/py/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'pandas'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
geomancer 1.1.0 requires geoalchemy2, which is not installed.
geomancer 1.1.0 requires loguru, which is not installed.
geomancer 1.1.0 requires google-cloud-bigquery[pandas,pyarrow]==1.9.0, but you have google-cloud-bigquery 3.3.5 which is incomp

In [ ]:
# references
# https://geoffboeing.com/2016/11/osmnx-python-street-networks/

In [3]:
#key: layername, value1: layer, value2: wrap
cities = {
    'CAVITE': '14.3294,120.9367',
    'ZAMBOANGA_CITY': '6.9167,122.0833',
    'LAGUNA': '14.2167,121.1667',
    'CEBU': '10.3,123.9',
    'ILOILO': '10.7167,122.5667',
    'QUEZON_CITY': '14.6333,121.0333',
    'PANGASINAN': '15.9281,120.3489',
    'BULACAN': '14.7928,120.8789',
    'NEGROS_OCCIDENTAL': '9.9833,122.8167',
    'BATANGAS': '13.75,121.05',
    'RIZAL': '14.5842,121.1763',
    'PAMPANGA': '15.0333,120.6833',
    'BUKIDNON': '7.9,125.0833',
    'NUEVA_ECIJA': '15.4833,120.9667',
    'TARLAC': '15.4802,120.5979',
    'NEGROS ORIENTAL': '9.3103,123.3081',
    'BOHOL': '9.65,123.85',
    'SOUTH_COTABATO': '6.5,124.85',
    'QUEZON': '7.7333,125.1',
}

def latlong_gen(city, diff):
    latlong = cities[city]
    lat_v, long_v = latlong.split(",")
    
    #define size of BBOX?
    LL_lat = round(float(lat_v) - diff,4)
    LL_long = round(float(long_v) - diff,4)
    UR_lat = round(float(lat_v) + diff, 4)
    UR_long = round(float(long_v) + diff, 4)

    bbox = [LL_lat, LL_long, UR_lat, UR_long] 
    
    return bbox

In [ ]:
import osmnx as ox
import pandas as pd
ox.config(log_console=True, use_cache=True)

amenities = [
        ['bar', 'biergarten', 'cafe', 'fast_food'],
        ['food_court', 'ice_cream', 'pub', 'restaurant'],
        ['college', 'driving_school', 'kindergarten'],
        ['language_school', 'library', 'toy_library'],
        ['training', 'music_school', 'school', 'university'],
        ['bicycle_parking', 'bicycle_repair_station', 'bicycle_rental'],
        ['boat_rental', 'boat_sharing', 'bus_station', 'car_rental'],
        ['car_sharing', 'car_wash', 'compressed_air'],
        ['vehicle_inspection', 'charging_station'],
        ['ferry_terminal', 'fuel', 'grit_bin'],
        ['motorcycle_parking', 'parking', 'parking_entrance'],
        ['parking_space', 'taxi', 'atm', 'bank'],
        ['bureau_de_change', 'baby_hatch', 'clinic'],
        ['dentist', 'doctors', 'hospital', 'nursing_home'],
        ['pharmacy', 'social_facility', 'veterinary'],
        ['arts_centre', 'brothel', 'casino', 'cinema'],
        ['community_centre', 'conference_centre', 'events_venue'],
        ['fountain', 'gambling', 'love_hotel', 'nightclub'],
        ['planetarium', 'public_bookcase', 'social_centre'],
        ['stripclub', 'studio', 'swingerclub', 'theatre'],
        ['courthouse', 'fire_station', 'police', 'post_box'],
        ['post_depot', 'post_office', 'prison', 'ranger_station'],
        ['townhall', 'bbq', 'bench', 'dog_toilet', 'dressing_room'],
        ['drinking_water', 'give_box', 'parcel_locker', 'shelter'],
        ['shower', 'telephone', 'toilets', 'water_point', 'watering_place'],
        ['sanitary_dump_station', 'recycling', 'waste_basket'],
        ['waste_disposal', 'waste_transfer_station', 'animal_boarding'],
        ['animal_breeding', 'animal_shelter', 'baking_oven', 'childcare'],
        ['clock', 'crematorium', 'dive_centre', 'funeral_hall', 'grave_yard'],
        ['hunting_stand', 'internet_cafe', 'kitchen', 'kneipp_water_cure'],
        ['lounger', 'marketplace', 'monastery', 'photo_booth'],
        ['place_of_mourning', 'place_of_worship', 'public_bath'],
        ['public_building', 'refugee_site', 'vending_machine']
]

for city, ltln in cities.items():
    gdf_arr = []

    for amenity in amenities:
        bbox = latlong_gen(city, 0.11) # [6.8067, 121.9733, 7.0267, 122.1933]

        north = bbox[0] #6.8067
        south = bbox[2] #7.0267 
        east = bbox[1] #121.9733
        west = bbox[3] #122.1933

        tags = {'amenity': amenity}
        print(city)
        gdf = ox.geometries_from_bbox(north, south, east, west, tags=tags)
        gdf_arr.append(gdf)
        print('successful', city, amenity)

    new_df = pd.concat(gdf_arr, axis=0, ignore_index=True)
    new_df.to_csv(city + ".csv")